In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os

sys.path.append(os.path.abspath("/data/home/ngjhn/read"))


In [ ]:
from coco import *
from adapter_utils import *
from utils import *

def print_memory(device, msg):  
        
    memory_reserved_client = round(
        torch.cuda.memory_reserved(device) / 1024**3, 3
    )
    memory_allocated_client = round(
        torch.cuda.memory_allocated(device) / 1024**3, 3
    )
    print(
        f"{msg}, cached/alloc memory at client device {device} was {memory_reserved_client} GB and {memory_allocated_client} GB"
    )


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!nvidia-smi

Mon Apr 24 19:57:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:90:1C.0 Off |                    0 |
| N/A   25C    P0    55W / 400W |  11465MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
data_module = CocoDataModule(
    data_dir="/data/home/ngjhn/read/coco",
    ann_dir="/data/home/ngjhn/read/coco/annotations/",
    vision_model_name_or_path="google/vit-base-patch16-224-in21k",
    split_file="/data/home/ngjhn/read/coco/dataset_coco.json",
    collate_fn=collate_fn,
    train_batch_size=8,
    val_batch_size=128)
data_module.setup()
batch = next(iter(data_module.train_dataloader()))

loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
loading annotations into memory...
Done (t=0.30s)
creating index...
index created!
loading annotations into memory...
Done (t=0.29s)
creating index...
index created!


In [8]:
!unset LD_LIBRARY_PATH

In [6]:


torch.cuda.reset_peak_memory_stats("cuda")

model = ImageCaptioningModel("google/vit-base-patch16-224-in21k",
                            "bert-large-uncased",
                            data_module=data_module,
                            learning_rate=1e-4,
                            peft_conf=create_config("rnn",
                                                    hidden_dim=256,
                                                    prefix_length=10))

# print_memory("cuda", "Before model.cuda()")
# model = model.cuda()
# opt = model.configure_optimizers()
# print_memory("cuda", "After model.cuda()")
# output = model.training_step(batch, 1)
# print_memory("cuda", "After training_step")
# output['loss'].backward()
# print_memory("cuda", "After backward")
# opt.step()
# print_memory("cuda", "After step")

# peak_memory = (torch.cuda.max_memory_allocated() / 1024**2) / 1000
# print(f"Memory utilization {peak_memory:.3f} GB")

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['bert.encoder.layer.20.crossattention.output.dense.bias', 'bert.encoder.layer.3.crossattention.output.dense.weight', 'bert.encoder.layer.23.crossattention.output.dense.bias', 'bert.encoder.layer.9.crossattention.self.value.weight', 'bert

Enables rnn-adapter tuning; num trainable params 2,234,880, ratio 0.005
loading annotations into memory...
Done (t=0.52s)
creating index...
index created!
loading annotations into memory...
Done (t=0.27s)
creating index...
index created!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!


In [9]:
with torch.no_grad():
    print_memory("cuda", "Before model.cuda()")
    model = model.cuda()
    images, image_ids = batch["pixel_values"], batch["image_id"]
    images = images.cuda()
    print_memory("cuda", "After model.cuda()")
    model.model.generate(images)
    print_memory("cuda", "After generate")
    peak_memory = (torch.cuda.max_memory_allocated() / 1024**2) / 1000
    print(f"Memory utilization {peak_memory:.3f} GB")

Before model.cuda(), cached/alloc memory at client device cuda was 0.373 GB and 0.325 GB
After model.cuda(), cached/alloc memory at client device cuda was 2.014 GB and 1.965 GB


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/transformers/generation/utils.py:1190: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/transformers/generation/utils.py:1288: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


After generate, cached/alloc memory at client device cuda was 2.566 GB and 2.028 GB
Memory utilization 2.457 GB
